In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from spectral_denoising.spectral_denoising import *
from spectral_denoising.chem_utils import *
smiles = 'O=c1nc[nH]c2nc[nH]c12'
adduct = '[M+Na]+'

peak = np.array([[48.992496490478516 ,154.0],
                  [63.006099700927734, 265.0],
                  [79.02062225341797, 521.0],
                  [159.02373146795, 999]], 
                  
                  dtype = np.float32)
computed_pmz = calculate_precursormz(adduct, smiles)
pmz, mass_threshold = get_pmz_statistics(peak, computed_pmz, mass_tolerance=0.005)
print(pmz, mass_threshold)

159.02373 0.006996155


In [13]:
master_formula = prep_formula(smiles, adduct)

In [14]:
all_possible_candidate_formula,all_possible_mass = get_all_subformulas(master_formula)

In [17]:
print(all_possible_candidate_formula[1:5], all_possible_mass[1:5])

['H', 'H2', 'H3', 'H4'] [1.00782503 2.01565006 3.0234751  4.03130013]


In [40]:
import numpy as np
import spectral_denoising as sd
from spectral_denoising.spectral_operations import *
from spectral_denoising.chem_utils import *

smiles = 'O=c1nc[nH]c2nc[nH]c12'
adduct = '[M+Na]+'
pmz = calculate_precursormz(adduct,smiles)
peak = np.array([[48.992496490478516 ,154.0],
                  [63.006099700927734, 265.0],
                  [79.02062225341797, 521.0]], dtype = np.float32)
print(f'the spectrum entropy is {spctrum_entropy(peak):.2f}, the normalized entropy is {normalized_entropy(peak):.2f}')
# alternatively, you can store mass and intensity in separate arrays, and use pack_spectrum(mass, intensity) to get the peaks array
# e.g.mass,intensity = [48.992496490478516, 63.006099700927734, 79.02062225341797], [154.0, 265.0, 521.0]
# peak = pack_spectrum(mass, intensity)

# generate some electronic noise and add it to the peaks
from spectral_denoising.noise import *
noise = generate_noise(pmz, lamda=10, n = 50)
peak_with_noise = add_noise(peak, noise)
# use head_to_tail_plot to visualize the spectra, only in jupyter notebook
# sd.head_to_tail_plot(peaks_with_noise,peaks ,pmz)
print(f'the spectrum entropy is {spctrum_entropy(peak_with_noise):.2f}, the normalized entropy is {normalized_entropy(peak_with_noise):.2f}')
print(f'the entropy similarity of contaminated spectrum and the raw spectrum is {entropy_similairty(peak_with_noise,peak,  pmz = pmz):.2f}')

# perform spectral denosing and compare against the raw spectrum
peak_denoised = sd.spectral_denoising(peak_with_noise, smiles, adduct)
print(f'the entropy similarity of denoised spectrum and the raw spectrum is {entropy_similairty(peak_denoised, peak, pmz = pmz):.2f}')
# use head_to_tail_plot to visualize the spectra, only in jupyter notebook
# sd.head_to_tail_plot(peaks_denoised,peaks ,pmz)

the spectrum entropy is 0.98, the normalized entropy is 0.63
the spectrum entropy is 3.67, the normalized entropy is 0.73
the entropy similarity of contaminated spectrum and the raw spectrum is 0.48
the entropy similarity of denoised spectrum and the raw spectrum is 0.92


In [41]:
import spectral_denoising as sd
query_spectra= sd.read_msp('sample_data/query_spectra.msp')
reference_library =sd.read_msp('sample_data/reference_library.msp')
query_spectrum, query_pmz = query_spectra.iloc[0]['peaks'], query_spectra.iloc[0]['precursor_mz'] # just the first spectrum
result = sd.denoising_search(query_spectrum, query_pmz, reference_library)
display(result)

/Users/fanzhoukong/Documents/GitHub/spectral_denoising/spectral_denoising/file_io.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[column] = pd.to_numeric(df[column], errors='ignore')


,name,peaks,precursor_mz,cas,adduct,mz,comment,smiles,formula,mix,...,reci_snr,ms2_scan_idx,pmz_offset,entropy,num peaks,entropy_similarity,denoised_similarity,quene_peaks,quene_pmz_denoised,quene_pmz
0,Inosine,"[[159.02952575683594, 1058.0], [291.0574951171...",291.069435,58-63-9,[M+Na]+,291.06999,Major,O=c1[nH]cnc2c1ncn2[C@@H]1O[C@H](CO)[C@@H](O)[C...,C10H12N4O5,Alkaloids-pos-mix1,...,0.004769,277,0.000659,1.0,2,0.591505,1.0,"[[55.05794906616211, 26.0], [60.98749923706055...","[[159.02952575683594, 1058.0], [291.0574951171...",291.069435


In [24]:
query_spectrum, query_pmz = query_spectra.iloc[0]['peaks'], query_spectra.iloc[0]['precursor_mz'] # just the first spectrum
results = sd.denoising_search_batch(query_spectra['peaks'], query_spectra['precursor_mz'], reference_library)


100%|██████████| 565/565 [00:10<00:00, 53.80it/s]


In [43]:
display(results[6])

,name,peaks,precursor_mz,cas,adduct,mz,comment,smiles,formula,mix,...,reci_snr,ms2_scan_idx,pmz_offset,entropy,num peaks,entropy_similarity,denoised_similarity,quene_peaks,quene_pmz_denoised,quene_pmz
420,"(R,S)-Anatabine","[[80.04740905761719, 757.0], [82.0651016235351...",161.104730,2743-90-0,[M+H]+,161.107325,isomer,C1=CCC(c2cccnc2)NC1,C10H12N2,Alkaloids-pos-mix34,...,0.000453,4141,0.000266,0.9999998807907104,9,0.638592,0.572749,"[[40.029239654541016, 62.0], [41.0396919250488...","[[41.03969192504883, 92.0], [51.02379989624023...",161.107275
419,"(R,S)-Anatabine","[[53.03861999511719, 258.0], [65.0372467041015...",161.105909,2743-90-0,[M+H]+,161.107325,Major,C1=CCC(c2cccnc2)NC1,C10H12N2,Alkaloids-pos-mix34,...,0.000167,179,0.001444,0.9999998807907104,25,0.732812,0.690817,"[[40.029239654541016, 62.0], [41.0396919250488...","[[41.03969192504883, 92.0], [51.02379989624023...",161.107275
6,Tryptamine,"[[41.03969192504883, 92.0], [51.02379989624023...",161.107275,61-54-1,[M+H]+,161.107325,Major,NCCc1c[nH]c2ccccc12,C10H12N2,Alkaloids-pos-mix1,...,0.002026,750,0.001936,0.9958412647247314,32,0.940968,1.000000,"[[40.029239654541016, 62.0], [41.0396919250488...","[[41.03969192504883, 92.0], [51.02379989624023...",161.107275


In [5]:
import spectral_denoising as sd
from spectral_denoising.noise import *
from spectral_denoising.spectral_operations import *
from spectral_denoising.chem_utils import *
peak = np.array([[48.992496490478516 ,154.0],
                [63.006099700927734, 265.0],
                [79.02062225341797, 521.0]], dtype = np.float32)
noise = generate_noise(pmz, lamda=10, n = 50)
smiles = 'O=c1nc[nH]c2nc[nH]c12'
adduct = '[M+Na]+'
pmz = calculate_precursormz(adduct,smiles)
peak_with_noise = add_noise(peak, noise)
peak_denoised = sd.spectral_denoising(peak_with_noise, smiles, adduct)
print(f'Entropy similarity of spectra with noise: {sd.entropy_similairty(peak_with_noise,peak, pmz ):.2f}.')
print(f'Entropy similarity of spectra with noise: {sd.entropy_similairty(peak_denoised,peak, pmz ):.2f}.')

Entropy similarity of spectra with noise: 0.45.
Entropy similarity of spectra with noise: 1.00.
